In [1]:
# Installing dependencies on Linux
!apt update -qq > /dev/null
# Installs Java 8 as Pyspark 2.4.4 still requires Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!rm -r spark*
!wget http://mirror.rise.ph/apache/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!ls
!tar xvf ./spark-2.4.8-bin-hadoop2.7.tgz > /dev/null 2>/dev/null
!ls
!pip install -q findspark

rm: cannot remove 'spark*': No such file or directory
--2022-04-21 16:54:48--  http://mirror.rise.ph/apache/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
Resolving mirror.rise.ph (mirror.rise.ph)... 43.226.6.79
Connecting to mirror.rise.ph (mirror.rise.ph)|43.226.6.79|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235899716 (225M) [application/x-gzip]
Saving to: ‘spark-2.4.8-bin-hadoop2.7.tgz’

spark-2.4.8-bin-had 100%[===================>] 224.97M  12.2MB/s    in 20s     

2022-04-21 16:55:09 (11.2 MB/s) - ‘spark-2.4.8-bin-hadoop2.7.tgz’ saved [235899716/235899716]

sample_data  spark-2.4.8-bin-hadoop2.7.tgz
sample_data  spark-2.4.8-bin-hadoop2.7	spark-2.4.8-bin-hadoop2.7.tgz


In [3]:
# Set environment variables
import os
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7/"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_312"
OpenJDK Runtime Environment (build 1.8.0_312-8u312-b07-0ubuntu1~18.04-b07)
OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)


In [21]:
import io
import findspark
findspark.init()

import re
import functools
from operator import add
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, Row, DataFrame

spark = SparkSession.builder.appName("assignment11").getOrCreate()


In [22]:
!rm -r googleplaystore.csv*
!rm -r googleplaystore_user_reviews.csv*
!ls

output.txt   spark-2.4.8-bin-hadoop2.7
sample_data  spark-2.4.8-bin-hadoop2.7.tgz


In [23]:
!wget -q https://raw.githubusercontent.com/andypilapil/assignment11/main/googleplaystore.csv
googleplaystore_df = spark.read.load('googleplaystore.csv', format='csv', sep = ',', header='true', escape='"', inferSchema='true')
!wget -q https://raw.githubusercontent.com/andypilapil/assignment11/main/googleplaystore_user_reviews.csv
googleplaystorereviews_df = spark.read.load('googleplaystore_user_reviews.csv', format='csv', sep = ',', header='true', escape='"', inferSchema='true').cache()

In [24]:
conf = SparkConf().setMaster("local").setAppName("assignment11")
sc = SparkContext.getOrCreate(conf = conf)

googleplaystorereviews_df_stripped = googleplaystorereviews_df.drop("Translated_Review", "Sentiment_Polarity", "Sentiment_Subjectivity")
googleplaystore_df_stripped = googleplaystore_df.drop("Category", "Reviews", "Size", "Installs", "Type", "Price", "Content Rating", "Genres", "Last Updated", "Current Ver", "Android Ver")

unioned_df = googleplaystore_df_stripped.join(googleplaystorereviews_df_stripped,['App'],how='inner')
unioned_df = unioned_df.select(["Rating","App","Sentiment"])
output = unioned_df.rdd.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y).sortBy(lambda x: x[0], ascending=False).collect()

with open('output.txt', 'w') as writefile:
  for (a, b) in output:
    if (str(a[0]) == "nan" or str(a[2]) == "nan"):
      continue
    else:
      writefile.write("Rating = " + str(a[0]) + ", App = " + str(a[1]) + ", Sentiment = " + str(a[2]) + ", Value = " + str(b) + "\n")


In [26]:
with open('output.txt', 'r') as testwritefile:
    print(testwritefile.read())

# Submitted by: Andrea Pilapil & Mari Valle

Rating = 4.9, App = Down Dog: Great Yoga Anywhere, Sentiment = Positive, Value = 40
Rating = 4.9, App = DMV Permit Practice Test 2018 Edition, Sentiment = Positive, Value = 30
Rating = 4.9, App = DMV Permit Practice Test 2018 Edition, Sentiment = Neutral, Value = 2
Rating = 4.9, App = DMV Permit Practice Test 2018 Edition, Sentiment = Negative, Value = 2
Rating = 4.9, App = CDL Practice Test 2018 Edition, Sentiment = Positive, Value = 21
Rating = 4.9, App = CDL Practice Test 2018 Edition, Sentiment = Neutral, Value = 2
Rating = 4.9, App = CDL Practice Test 2018 Edition, Sentiment = Negative, Value = 1
Rating = 4.8, App = Home Workout for Men - Bodybuilding, Sentiment = Positive, Value = 22
Rating = 4.8, App = Home Workout for Men - Bodybuilding, Sentiment = Neutral, Value = 4
Rating = 4.8, App = Home Workout - No Equipment, Sentiment = Positive, Value = 48
Rating = 4.8, App = Home Workout - No Equipment, Sentiment = Neutral, Value = 2
Rating = 4.8, App = Home Workout - No Equipment, Se